In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from gwpy.timeseries import TimeSeries
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from lib.fromACC2 import MassPoint, GWStrainCalculator
from matplotlib.patches import Circle

In [2]:
"""
Importing data from our tracked objects from blender, merging both csv's into one dataframe. We're chosing to merge on "Left" by convention, 
but if the second data set is shorter than the first then you may want to merge on "Right".
"""

data2 = pd.read_csv('blender/data/IMG_3105_Blue_Track.csv', header=None, names=["Time", "mass1_x", "mass1_y"])
data3 = pd.read_csv('blender/data/IMG_3105_Red_Track.csv', header=None, names=["Time", "mass2_x", "mass2_y"])
data = pd.merge(data2, data3, on="Time", how="left")

In [3]:
"""
Converting the dataframe into our own MassPoint class, calculating the GW strain and appending the strain data to their own arrays for plotting.
Note: with theta and phi set to zero we expect non-zero values for only h_plus. Vary these as needed.
"""

# STRAIN AMPLITUDE CALCS

trajectory = []

for _, row in data.iterrows():
    # Create MassPoint objects for each mass
    mass1_point = MassPoint(mass=25, x=row['mass1_x'], y=row['mass1_y'])
    mass2_point = MassPoint(mass=30, x=row['mass2_x'], y=row['mass2_y'])
    
    # Append MassPoint positions to trajectory []
    trajectory.append([mass1_point, mass2_point])

# Set up the strain calculator with parameters
calculator = GWStrainCalculator()
r_observer = 1e20  # 10 kpc
theta = 0          # Overhead observer
phi = 0
dt = data['Time'].iloc[1] - data['Time'].iloc[0]  # Calculate the time step from the CSV

# Calculate strain for each timestep pair
H_plus = []
H_cross = []
for i in range(1, len(trajectory) - 1):
    masses_t1 = trajectory[i - 1]  # Previous time step
    masses_t2 = trajectory[i]      # Current time step
    masses_t3 = trajectory[i + 1]  # Next time step
    
    h_plus, h_cross = calculator.calculate_strain_components(
        masses_t1, masses_t2, masses_t3, dt, r_observer, theta, phi
    )
    H_plus.append(h_plus)
    H_cross.append(h_cross)

In [4]:
# Create main figure and axis for the orbit animation
fig, ax = plt.subplots(figsize=(12, 12))
fig.patch.set_facecolor('lightgrey')   # Figure background
ax.set_facecolor('lightgrey')          # Axes background
ax.axis('off')
ax.set_aspect('equal', adjustable='datalim')

time = data['Time'][:-2]

# Set margins
margin_factor = 1.1
x_min, x_max = min(data['mass1_x'].min(), data['mass2_x'].min()), max(data['mass1_x'].max(), data['mass2_x'].max())
y_min, y_max = min(data['mass1_y'].min(), data['mass2_y'].min()), max(data['mass1_y'].max(), data['mass2_y'].max())
x_margin = (x_max - x_min) * (margin_factor - 1)
y_margin = (y_max - y_min) * (margin_factor - 1)

# Create background patch
back = plt.Rectangle((x_min - x_margin, y_min - y_margin), 
                     (x_max - x_min) + 2 * x_margin, 
                     (y_max - y_min) + 2 * y_margin, 
                     color="lightgrey")

# Create stars and trajectories
star_1 = Circle((data['mass1_x'][0], data['mass1_y'][0]), 5, color='blue')
star_2 = Circle((data['mass2_x'][0], data['mass2_y'][0]), 5, color='red')
trajectory1, = ax.plot([], [], linestyle='dotted', color='blue', linewidth=1)
trajectory2, = ax.plot([], [], linestyle='dotted', color='red', linewidth=1)

ax.add_patch(back)
ax.add_patch(star_1)
ax.add_patch(star_2)
trajectory1.set_data([], [])
trajectory2.set_data([], [])

# Inset axis for strain plot
inset_ax = fig.add_axes([0.70, 0.025, 0.25, 0.25], facecolor='white')
inset_ax.set_xlim(0, max(time))
inset_ax.set_ylim(
    min(np.min(H_plus), np.min(H_cross)) * 1.1, 
    max(np.max(H_plus), np.max(H_cross)) * 1.1
)
inset_ax.tick_params(colors='black')              
inset_ax.spines[:].set_color('black')             
inset_ax.xaxis.label.set_color('black')           
inset_ax.yaxis.label.set_color('black')
inset_ax.set_xlabel('Time')
inset_ax.set_ylabel('Strain Amplitude')
inset_ax.grid(True, color='black', linestyle=':', linewidth=0.5)

# Plot both waveforms
waveform_plus, = inset_ax.plot([], [], linestyle='-', color='blue', label='$h_+$')
waveform_cross, = inset_ax.plot([], [], linestyle='-', color='red', label='$h_\\times$')
inset_ax.legend(loc='upper right', facecolor='white', edgecolor='black')

# Update function
def update(frame):
    # Orbit update
    star_1.center = (data['mass1_x'][frame], data['mass1_y'][frame])
    star_2.center = (data['mass2_x'][frame], data['mass2_y'][frame])
    trajectory1.set_data(data['mass1_x'][:frame+1], data['mass1_y'][:frame+1])
    trajectory2.set_data(data['mass2_x'][:frame+1], data['mass2_y'][:frame+1])
    
    # Strain update
    time_data = time[:frame+1]
    waveform_plus.set_data(time_data, H_plus[:frame+1])
    waveform_cross.set_data(time_data, H_cross[:frame+1])
    
    return star_1, star_2, trajectory1, trajectory2, waveform_plus, waveform_cross

# Create animation
animation = FuncAnimation(fig, update, frames=len(time), interval=30, blit=True, save_count=0)

plt.close()
# HTML(animation.to_jshtml()) # enable to show animation in notebook
animation.save('BlueRedDots.gif', writer='pillow', fps=20) # enable to save animation as gif


C:\Users\Diego\AppData\Local\Temp\ipykernel_2948\2099527860.py:71: UserWarning: You passed in an explicit save_count=0 which is being ignored in favor of frames=653.
  animation = FuncAnimation(fig, update, frames=len(time), interval=30, blit=True, save_count=0)


In [5]:
# DARK MODE

# Create main figure and axis for the orbit animation
fig, ax = plt.subplots(figsize=(12, 12))
fig.patch.set_facecolor('black')
ax.set_facecolor('#121212')
ax.axis('off')
ax.set_aspect('equal', adjustable='datalim')

time = data['Time'][:-2]

# Create background and stars for the orbit
margin_factor = 1.1  # 10% margin

x_min, x_max = min(data['mass1_x'].min(), data['mass2_x'].min()), max(data['mass1_x'].max(), data['mass2_x'].max())
y_min, y_max = min(data['mass1_y'].min(), data['mass2_y'].min()), max(data['mass1_y'].max(), data['mass2_y'].max())

x_margin = (x_max - x_min) * (margin_factor - 1)
y_margin = (y_max - y_min) * (margin_factor - 1)

back = plt.Rectangle(
    (x_min - x_margin, y_min - y_margin),
    (x_max - x_min) + 2 * x_margin,
    (y_max - y_min) + 2 * y_margin,
    color="#121212"
)

star_1 = Circle((data['mass1_x'][0], data['mass1_y'][0]), 5, color='blue')
star_2 = Circle((data['mass2_x'][0], data['mass2_y'][0]), 5, color='red')
trajectory1, = ax.plot([], [], linestyle='dotted', color='blue', linewidth=1)
trajectory2, = ax.plot([], [], linestyle='dotted', color='red', linewidth=1)

# Add background and stars
ax.add_patch(back)
ax.add_patch(star_1)
ax.add_patch(star_2)
trajectory1.set_data([], [])
trajectory2.set_data([], [])

# Create a subspace for the strain amplitude animation
inset_ax = fig.add_axes([0.70, 0.025, 0.25, 0.25], facecolor='black')  # inset: bottom right corner
inset_ax.set_xlim(0, max(time))
inset_ax.set_ylim(
    min(np.min(H_plus), np.min(H_cross)) * 1.1,
    max(np.max(H_plus), np.max(H_cross)) * 1.1
)
inset_ax.tick_params(colors='white')
inset_ax.spines[:].set_color('white')
inset_ax.xaxis.label.set_color('white')
inset_ax.yaxis.label.set_color('white')
inset_ax.set_xlabel('Time')
inset_ax.set_ylabel('Strain Amplitude')

# Create two waveforms for h+ and h×
waveform_plus, = inset_ax.plot([], [], linestyle='-', color='blue', label='$h_+$')
waveform_cross, = inset_ax.plot([], [], linestyle='-', color='red', label='$h_\\times$')

# Add legend
inset_ax.legend(loc='upper right', fontsize=8, facecolor='black', labelcolor='white')

# Update function for both animations
def update(frame):
    # Update orbit animation
    star_1.center = (data['mass1_x'][frame], data['mass1_y'][frame])
    star_2.center = (data['mass2_x'][frame], data['mass2_y'][frame])
    trajectory1.set_data(data['mass1_x'][:frame+1], data['mass1_y'][:frame+1])
    trajectory2.set_data(data['mass2_x'][:frame+1], data['mass2_y'][:frame+1])
    
    # Update strain animation in the inset
    time_data = time[:frame+1]
    H_PLUS = H_plus[:frame+1]
    H_CROSS = H_cross[:frame+1]
    waveform_plus.set_data(time_data, H_PLUS)
    waveform_cross.set_data(time_data, H_CROSS)
    
    return star_1, star_2, trajectory1, trajectory2, waveform_plus, waveform_cross

# Create animation
animation = FuncAnimation(fig, update, frames=len(time), interval=30, blit=True, save_count=0)

plt.close()  # Prevent duplicate display of the animation
plt.rcParams['animation.html'] = 'html5'
plt.rcParams['animation.embed_limit'] = 1000

# HTML(animation.to_jshtml()) # enable to show animation in notebook
animation.save('BlueRedDots_DARK.gif', writer='pillow', fps=20) # enable to save animation as gif


C:\Users\Diego\AppData\Local\Temp\ipykernel_2948\1541473690.py:79: UserWarning: You passed in an explicit save_count=0 which is being ignored in favor of frames=653.
  animation = FuncAnimation(fig, update, frames=len(time), interval=30, blit=True, save_count=0)
